# AffNet Results

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:,.2f}".format

In [31]:
def extract_metrics_gpu(filename): 
    f = open(filename, 'r')
    lines = f.readlines() 
    ram_init = None
    ram_peak = None
    cpu_init = None 
    cpu_end = None 
    for line in lines: 
        if "=> Average precision" in line: 
            accuracy = float(line.split(" ")[-1])
        if "=> Total training time" in line: 
            training_time = float(line.split(" ")[-2])
        if "RAM BEFORE TRAINING:" in line: 
            ram_init = float(line.split(" ")[-1])
        if "PEAK TRAINING RAM:" in line: 
            ram_peak = float(line.split(" ")[-1])
        if "CPU BEFORE TRAINING:" in line: 
            cpu_init = eval(line.split(":")[-1])
        if "CPU BEFORE EVALUATION:" in line: 
            cpu_end = eval(line.split(":")[-1])
        if "GPU BEFORE EVALUATION:" in line:
            gpu_u, _ = eval(line.split(":")[-1])
        if "Peak mem and init mem:" in line:
            gpu_mem = float(line.split(" ")[-2])
        
    cpu_u, cpu_k = (cpu_end[0] - cpu_init[0], cpu_end[1] - cpu_init[1])
    return (accuracy * 100, training_time, (ram_peak-ram_init) * 1024, cpu_u + cpu_k, gpu_u, gpu_mem)

In [32]:
def calculate_metrics_gpu(files): 
    metrics = dict()
    
    for file in files: 
        method, seed, _ = file.split("_") 
        
        if method in metrics: 
            metrics[method].append(extract_metrics_gpu(file))
        else: 
            metrics[method] = [extract_metrics_gpu(file)] 
    
    return metrics

In [33]:
def get_means(metrics): 
    return {method: [sum(x)/len(x) for x in list(zip(*metrics[method]))] for method in metrics}

In [34]:
def get_sds(metrics): 
    return {method: [np.std(x) for x in list(zip(*metrics[method]))] for method in metrics}

# 1. AlexNet Task-IL Results 

In [35]:
%cd /Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/affectnet/2022-06-27-10-57

/Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/affectnet/2022-06-27-10-57


In [36]:
logs = !ls
logs = [x for x in logs if "ALEXNET" in x]

In [37]:
ckp_metrics = calculate_metrics_gpu(logs)
ckp_means = get_means(ckp_metrics)
ckp_sds = get_sds(ckp_metrics)

In [38]:
df = pd.DataFrame(ckp_means, index=['Accuracy (%)',
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'Peak GPU Memory (MB)'])
df = df.filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

In [39]:
df

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),67.38,65.25,65.22,63.32,69.07,67.24
Training time (s),"4,832.23","4,860.53","5,309.63","4,809.50","5,105.90","4,947.90"
RAM Usage (MB),245.00,89.70,92.84,68.11,89.34,82.33
CPU Usage (s),"4,975.21","5,005.71","5,455.58","4,951.63","5,249.81","5,093.02"
GPU Usage (%),1.54,1.07,7.31,1.29,7.34,1.31
Peak GPU Memory (MB),"5,217.33","1,818.00","5,836.00","1,542.00","5,836.00","1,542.00"


In [40]:
print(df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &       nr &       lr &       gr &      lgr &      grd &     lgrd \\
\midrule
Accuracy (\%)         &    67.38 &    65.25 &    65.22 &    63.32 &    69.07 &    67.24 \\
Training time (s)    & 4,832.23 & 4,860.53 & 5,309.63 & 4,809.50 & 5,105.90 & 4,947.90 \\
RAM Usage (MB)       &   245.00 &    89.70 &    92.84 &    68.11 &    89.34 &    82.33 \\
CPU Usage (s)        & 4,975.21 & 5,005.71 & 5,455.58 & 4,951.63 & 5,249.81 & 5,093.02 \\
GPU Usage (\%)        &     1.54 &     1.07 &     7.31 &     1.29 &     7.34 &     1.31 \\
Peak GPU Memory (MB) & 5,217.33 & 1,818.00 & 5,836.00 & 1,542.00 & 5,836.00 & 1,542.00 \\
\bottomrule
\end{tabular}



# 2. MobileNet Task-IL Results 

In [41]:
%cd /Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/affectnet/2022-06-27-10-57

/Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/affectnet/2022-06-27-10-57


In [42]:
logs = !ls
logs = [x for x in logs if "MOBILENET-V2" in x]

In [43]:
ckp_metrics = calculate_metrics_gpu(logs)
ckp_means = get_means(ckp_metrics)
ckp_sds = get_sds(ckp_metrics)

In [44]:
df = pd.DataFrame(ckp_means, index=['Accuracy (%)',
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'Peak GPU Memory (MB)'])

df = df.filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

In [45]:
df

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),59.76,59.23,53.06,53.37,53.38,59.41
Training time (s),"5,013.67","5,062.03","5,480.90","4,974.10","5,403.23","5,016.10"
RAM Usage (MB),219.04,338.72,86.55,92.30,88.13,87.28
CPU Usage (s),"5,158.61","5,209.47","5,631.19","5,117.24","5,553.72","5,160.64"
GPU Usage (%),4.04,2.54,10.42,2.72,10.95,2.60
Peak GPU Memory (MB),"4,919.33","1,676.67","5,414.00","1,604.00","5,414.00","1,604.00"


In [46]:
print(df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &       nr &       lr &       gr &      lgr &      grd &     lgrd \\
\midrule
Accuracy (\%)         &    59.76 &    59.23 &    53.06 &    53.37 &    53.38 &    59.41 \\
Training time (s)    & 5,013.67 & 5,062.03 & 5,480.90 & 4,974.10 & 5,403.23 & 5,016.10 \\
RAM Usage (MB)       &   219.04 &   338.72 &    86.55 &    92.30 &    88.13 &    87.28 \\
CPU Usage (s)        & 5,158.61 & 5,209.47 & 5,631.19 & 5,117.24 & 5,553.72 & 5,160.64 \\
GPU Usage (\%)        &     4.04 &     2.54 &    10.42 &     2.72 &    10.95 &     2.60 \\
Peak GPU Memory (MB) & 4,919.33 & 1,676.67 & 5,414.00 & 1,604.00 & 5,414.00 & 1,604.00 \\
\bottomrule
\end{tabular}



# 3. ResNet Task-IL Results 

In [47]:
%cd /Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/affectnet/2022-06-27-10-57

/Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/affectnet/2022-06-27-10-57


In [48]:
logs = !ls
logs = [x for x in logs if "RESNET-18" in x]

In [49]:
ckp_metrics = calculate_metrics_gpu(logs)
ckp_means = get_means(ckp_metrics)
ckp_sds = get_sds(ckp_metrics)

In [50]:
df = pd.DataFrame(ckp_means, index=['Accuracy (%)',
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'Peak GPU Memory (MB)'])

df = df.filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

In [51]:
df

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),61.36,61.84,52.69,53.71,56.26,60.83
Training time (s),"5,068.93","4,884.23","5,272.73","4,995.77","5,271.57","4,955.30"
RAM Usage (MB),492.77,85.84,86.56,83.60,88.27,80.14
CPU Usage (s),"5,218.62","5,029.07","5,417.99","5,136.38","5,415.59","5,095.95"
GPU Usage (%),3.00,2.22,9.76,2.35,10.27,2.22
Peak GPU Memory (MB),"4,990.67","1,578.67","5,248.00","1,554.00","5,248.00","1,554.00"


In [52]:
print(df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &       nr &       lr &       gr &      lgr &      grd &     lgrd \\
\midrule
Accuracy (\%)         &    61.36 &    61.84 &    52.69 &    53.71 &    56.26 &    60.83 \\
Training time (s)    & 5,068.93 & 4,884.23 & 5,272.73 & 4,995.77 & 5,271.57 & 4,955.30 \\
RAM Usage (MB)       &   492.77 &    85.84 &    86.56 &    83.60 &    88.27 &    80.14 \\
CPU Usage (s)        & 5,218.62 & 5,029.07 & 5,417.99 & 5,136.38 & 5,415.59 & 5,095.95 \\
GPU Usage (\%)        &     3.00 &     2.22 &     9.76 &     2.35 &    10.27 &     2.22 \\
Peak GPU Memory (MB) & 4,990.67 & 1,578.67 & 5,248.00 & 1,554.00 & 5,248.00 & 1,554.00 \\
\bottomrule
\end{tabular}



# 4. VGG-16 Task-IL Results 

In [53]:
%cd /Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/affectnet/2022-06-27-10-57

/Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/affectnet/2022-06-27-10-57


In [54]:
logs = !ls
logs = [x for x in logs if "VGG-16" in x]

In [55]:
ckp_metrics = calculate_metrics_gpu(logs)
ckp_means = get_means(ckp_metrics)
ckp_sds = get_sds(ckp_metrics)

In [56]:
df = pd.DataFrame(ckp_means, index=['Accuracy (%)',
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'Peak GPU Memory (MB)'])

df = df.filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

In [57]:
df

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),67.46,67.50,67.46,66.63,69.41,69.34
Training time (s),"5,507.53","5,108.70","6,154.00","5,435.43","6,124.80","5,410.70"
RAM Usage (MB),157.64,72.43,97.69,69.23,116.86,60.47
CPU Usage (s),"5,651.47","5,251.80","6,272.37","5,583.11","6,274.24","5,557.61"
GPU Usage (%),12.43,8.87,21.18,8.38,20.37,8.59
Peak GPU Memory (MB),"7,581.33","4,310.00","8,204.00","4,154.00","8,204.00","4,154.00"


In [58]:
print(df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &       nr &       lr &       gr &      lgr &      grd &     lgrd \\
\midrule
Accuracy (\%)         &    67.46 &    67.50 &    67.46 &    66.63 &    69.41 &    69.34 \\
Training time (s)    & 5,507.53 & 5,108.70 & 6,154.00 & 5,435.43 & 6,124.80 & 5,410.70 \\
RAM Usage (MB)       &   157.64 &    72.43 &    97.69 &    69.23 &   116.86 &    60.47 \\
CPU Usage (s)        & 5,651.47 & 5,251.80 & 6,272.37 & 5,583.11 & 6,274.24 & 5,557.61 \\
GPU Usage (\%)        &    12.43 &     8.87 &    21.18 &     8.38 &    20.37 &     8.59 \\
Peak GPU Memory (MB) & 7,581.33 & 4,310.00 & 8,204.00 & 4,154.00 & 8,204.00 & 4,154.00 \\
\bottomrule
\end{tabular}

